In [1]:
import pandas as pd
from dateutil import parser
import numpy as np

In [2]:
btc_usdt = pd.read_csv("./edav_onchaindata/BTC-USDT.csv")
btc_usdt["time"] = pd.to_datetime(btc_usdt["time"])
btc_usdt = btc_usdt.sort_values(by = "time")
# btc_usdt["price"] = (btc_usdt["high"].to_numpy() + btc_usdt["low"].to_numpy() + btc_usdt["close"].to_numpy())/3
btc_usdt["price"] =  btc_usdt["close"].to_numpy()
btc_usdt = btc_usdt[["time", "price"]]

In [3]:
fear_df = pd.read_csv("./edav_onchaindata/fundamentalDf.csv")
fear_df["time"] = pd.to_datetime(fear_df["time"]).dt.date
fear_df = fear_df[["fear", "time"]]

In [10]:
future_dfs

{datetime.date(2022, 3, 25):           time  future_price    price  basis_rate  recal_basis_rate  fear
 0   2021-09-25       44334.5  42678.0    3.881391         -1.548609    28
 1   2021-09-26       44892.0  43172.1    3.983823         -1.416177    27
 2   2021-09-27       43740.1  42152.9    3.765340         -1.604660    26
 3   2021-09-28       42490.3  41018.6    3.587885         -1.752115    25
 4   2021-09-29       43049.6  41527.4    3.665532         -1.644468    24
 ..         ...           ...      ...         ...               ...   ...
 62  2021-11-26       55531.2  53739.6    3.333854         -0.236146    47
 63  2021-11-27       56615.8  54726.8    3.451691         -0.088309    21
 64  2021-11-28       59519.8  57286.8    3.897931          0.387931    27
 65  2021-11-29       59978.7  57769.2    3.824702          0.344702    33
 66  2021-11-30       59149.3  56949.9    3.861991          0.411991    40
 
 [67 rows x 6 columns],
 datetime.date(2020, 6, 26):            time  

In [4]:
from os import listdir
from os.path import isfile, join
futures = [f for f in listdir("./edav_onchaindata/future-swap") if isfile(join("./edav_onchaindata/future-swap", f))]

In [5]:
future_dfs = {}
for file_name in futures:
    delvery_time = "20" + file_name[9:11] + "-" + file_name[11:13] + "-" + file_name[13:15]
    delvery_time = parser.parse(delvery_time).date()
    delvery_time = delvery_time
    df = pd.read_csv("./edav_onchaindata/future-swap/{}".format(file_name))
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values(by = "time")
#     df["future_price"] = (df["high"].to_numpy() + df["low"].to_numpy() + df["close"].to_numpy())/3
    df["future_price"] = df["close"].to_numpy()
    df = df[["time", "future_price"]]
    df = pd.merge(df, btc_usdt, how = "inner", left_on="time", right_on="time")
    df["basis_rate"] = (df["future_price"] - df["price"])/df["price"] * 100
    df["time"] = df["time"].dt.date
    df = df.groupby("time").tail(1)
    default_funding_rate = np.array((delvery_time - df.time) / np.timedelta64(1, 'D')) * 0.03
    df["recal_basis_rate"] = df["basis_rate"].to_numpy() - default_funding_rate
    df = df.iloc[2:len(df)-3]
    df = pd.merge(df, fear_df, how = "inner", left_on="time", right_on="time")
    future_dfs[delvery_time] = df

In [6]:
deliveries = list(future_dfs.keys())

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=8, cols=1, 
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}]])

for i in range(4):
    for j in range(2):
        df = future_dfs[deliveries[2*i+j]]
        fig.add_trace(
            go.Scatter(x=df.time, y=df.recal_basis_rate, name="recal_basis_rate"),
            row=2*i+j+1, col=1,
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(x=df.time, y=df.fear, name = "fear"),
            row=2*i+j+1, col=1,
            secondary_y=True,
        )
fig.update_layout(height=2400, width=900)
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
df = future_dfs[deliveries[0]]
df["fear "]

In [ ]:
np.array((datetime.datetime(2020, 6, 26, 0, 0) - B.index) / np.timedelta64(1, 'D')) * 0.3

In [ ]:
def fuck(X, shares):
    X_dic = {}
    for x in X:
        